In [38]:
import pandas as pd
from rembg import remove
import requests
import io
from PIL import Image


In [40]:
planilha = pd.read_excel("produtos_marcas_site.xlsx")
display(planilha)

,ID,COD,Descrição,Unidade,Classificação fiscal,Origem,Preço,Valor IPI fixo,Observações,Situação,...,Dias para preparação,Controlar lotes,Unidade por caixa,URL imagem externa 7,URL imagem externa 8,URL imagem externa 9,URL imagem externa 10,Markup,Permitir inclusão nas vendas,EX TIPI
0,763089936,29,ACETONA 100 AZUL AVANTE COD. 29,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",2.74,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1,763089941,253,ACETONA 100 ML COD. 253 ROSA AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",2.74,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
2,763089946,257,ACETONA 500 ML COD. 257 ROSA AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",12.30,0,NaN,Ativo,...,0,Sim,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
3,763089951,89,ACETONA 500ML ML COD. 89 AZUL AVANT,UN,2914.11.00,"0 - Nacional, exceto as indicadas nos códigos ...",12.30,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
4,763090027,ALGODÃO 100,ALGODÃO HIDROFILO NATHY 100G BOLA,UN,5203.00.00,"0 - Nacional, exceto as indicadas nos códigos ...",4.51,0,NaN,Ativo,...,0,Sim,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,806799955,GLOSSY ME NUDE,GLOSSY ME - BRILHO LABIAL NUDE/ BEGE OCEANE,NaN,NaN,"0 - Nacional, exceto as indicadas nos códigos ...",23.20,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1357,800850077,"BENIG PRETO 1,5","HENNA PRETO 1,5G LA BENIG",UN,1211.90.90,"0 - Nacional, exceto as indicadas nos códigos ...",10.90,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,0.00,Sim,NaN
1358,801063854,LIXA BLOCO ROSA C/10,LIXA BLOCO DE UNHA ROSA COM 10 UNI,PACOTE,NaN,"0 - Nacional, exceto as indicadas nos códigos ...",24.00,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,1.60,Sim,NaN
1359,799865059,NaN,MINI KIT DE PINCEIS,UN,9603.30.00,"0 - Nacional, exceto as indicadas nos códigos ...",0.00,0,NaN,Inativo,...,0,Não,NaN,NaN,NaN,NaN,NaN,1.55,Sim,NaN


In [37]:
import cv2
import numpy as np
import pandas as pd
import requests
import io
from PIL import Image, ImageEnhance
from rembg import remove

In [41]:
import pandas as pd
import requests
import io
from PIL import Image
import cv2
import numpy as np

# Função para remover ruído usando filtro bilateral
def remove_noise(image):
    cv_image = np.array(image.convert('RGB'))  # Converter imagem para formato numpy RGB
    bilateral_filtered_image = cv2.bilateralFilter(cv_image, 5, 75, 75)  # Filtro bilateral
    return bilateral_filtered_image

# Função para remover o fundo da imagem
def remove_background(image):
    # Converter a imagem para o formato numpy array
    cv_image = np.array(image)
    
    # Converter imagem para escala de cinza
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    
    # Aplicar threshold para obter uma máscara binária
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
    
    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Criar uma máscara de fundo vazio
    mask = np.zeros(cv_image.shape[:2], dtype=np.uint8)
    
    # Preencher o contorno maior encontrado
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    
    # Aplicar a máscara à imagem
    fg = cv2.bitwise_and(cv_image, cv_image, mask=mask)
    
    # Adicionar um canal alfa para a transparência
    bgra = cv2.cvtColor(fg, cv2.COLOR_BGR2BGRA)
    
    # Definir os pixels de fundo como transparentes
    bgra[:, :, 3] = mask
    
    return bgra

# Iterar sobre cada linha da planilha
for index, row in planilha.iterrows():
    nome_produto = str(row['COD'])  # Converter para string se não estiver
    url_imagem = row['URL']
    
    # Verificar se a URL é válida
    if pd.isna(url_imagem) or not isinstance(url_imagem, str) or not url_imagem.startswith(('http://', 'https://')):
        print(f"URL inválida para o produto '{nome_produto}': {url_imagem}")
        continue
    
    # Fazer o download da imagem
    response = requests.get(url_imagem)
    
    try:
        # Verificar se a resposta da requisição foi bem-sucedida
        if response.status_code == 200:
            # Criar um objeto de imagem a partir da imagem baixada
            image = Image.open(io.BytesIO(response.content))
            
            # Remover ruído da imagem
            image_enhanced = remove_noise(image)
            
            # Remover o fundo da imagem
            image_without_background = remove_background(image_enhanced)
            
            # Salvar a imagem com o nome do produto como PNG
            nome_arquivo = f"{nome_produto}.png"
            Image.fromarray(image_without_background).save(nome_arquivo, format="PNG")
            
            # Resultado:
            print(f"Imagem melhorada e salva como {nome_arquivo}")
        else:
            print(f"Não foi possível baixar a imagem para o produto '{nome_produto}': Código de status {response.status_code}")
    
    except Exception as e:
        print(f"Não foi possível baixar e melhorar a imagem para o produto '{nome_produto}': {e}")

Imagem melhorada e salva como 29.png
Imagem melhorada e salva como 253.png
Imagem melhorada e salva como 257.png
Imagem melhorada e salva como 89.png
Imagem melhorada e salva como ALGODÃO 100.png
Imagem melhorada e salva como ALGODÃO 250.png
Imagem melhorada e salva como ALGODÃO 25.png
Imagem melhorada e salva como ALGODÃO 25 CX.png
Imagem melhorada e salva como ALGODÃO 500.png
Imagem melhorada e salva como ALGODÃO 50.png
Imagem melhorada e salva como ALGODÃO 50 CX.png
Imagem melhorada e salva como 11064.png
Imagem melhorada e salva como 11063.png
Imagem melhorada e salva como 433.png
Imagem melhorada e salva como 432.png
Imagem melhorada e salva como 434..png
Imagem melhorada e salva como 376.png
Imagem melhorada e salva como DISCO 50.png
Imagem melhorada e salva como DISCO 100.png
Imagem melhorada e salva como 11042.png
Imagem melhorada e salva como 11041.png
Imagem melhorada e salva como 14027.png
Imagem melhorada e salva como 37.png
Imagem melhorada e salva como OLEO BANANA 367.png

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Imagem melhorada e salva como 33.png
Imagem melhorada e salva como 80.png
Imagem melhorada e salva como 213.png
Imagem melhorada e salva como 223.png
Imagem melhorada e salva como 167.png
Imagem melhorada e salva como MCLK332.png
Imagem melhorada e salva como MCLK339.png
Imagem melhorada e salva como MCL3011.png
Imagem melhorada e salva como MCL3021.png
Imagem melhorada e salva como MCL3027.png
Imagem melhorada e salva como CC-6103.png
Imagem melhorada e salva como CC-6105.png
Imagem melhorada e salva como CC-6107.png
Imagem melhorada e salva como CE 022.png
Imagem melhorada e salva como CE 023.png
Imagem melhorada e salva como 204.png
Imagem melhorada e salva como MAC239.png
Imagem melhorada e salva como MAC245.png
Imagem melhorada e salva como MAC268.png
Imagem melhorada e salva como MAC276.png
Imagem melhorada e salva como 303.png
Imagem melhorada e salva como MAC217.png
Imagem melhorada e salva como MAC 282.png
Imagem melhorada e salva como 289.png
Imagem melhorada e salva como MAC

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))